# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

1. Import libraries.

In [3]:
from pandas.io.json import json_normalize
import pandas as pd
import json
import requests

2. Define the initial API endpoint URL.

In [4]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
initial_endpoint = 'https://www.zalando.fr/api/catalog/articles?categories=promo-sport-femme&limit=84&offset=0&sort=sale'


3. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

In [5]:
flattened_data = pd.DataFrame()
response = requests.get(initial_endpoint,headers=headers)
results = response.json()
results

{'total_count': 6323,
 'pagination': {'page_count': 76, 'current_page': 1, 'per_page': 84},
 'sort': 'sale',
 'articles': [{'sku': 'ON541F03U-C11',
   'name': 'MANEUVER INSULATOR JACKET - Veste de snowboard - gun metal',
   'price': {'original': '169,99\xa0€',
    'promotional': '101,99\xa0€',
    'has_different_prices': False,
    'has_different_original_prices': False,
    'has_different_promotional_prices': False,
    'has_discount_on_selected_sizes_only': False},
   'sizes': ['S', 'M', 'L', 'XL'],
   'url_key': 'oneill-maneuver-insulator-jacket-veste-de-ski-on541f03u-c11',
   'media': [{'path': 'ON/54/1F/03/UC/11/ON541F03U-C11@9.jpg',
     'role': 'DEFAULT',
     'packet_shot': False},
    {'path': 'ON/54/1F/03/UC/11/ON541F03U-C11@14.jpg',
     'role': 'HOVER',
     'packet_shot': False}],
   'brand_name': "O'Neill",
   'is_premium': False,
   'family_articles': [{'sku': 'ON541F03U-C11',
     'url_key': 'oneill-maneuver-insulator-jacket-veste-de-ski-on541f03u-c11',
     'media': [{

4. Find out the total page count in the 1st page data.

In [6]:
pages_count = results['pagination']['page_count']
pages_count

76

5. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

6. Print and review the data you obtained.

## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [ ]:
# your code here